# Minimal Example pandapower


## Creating a Power System

We consider the following simple 3-bus example network as a minimal example:

<img src="pics/3bus-system.png" width="50%">

The above network can be created in pandapower as follows:

In [ ]:
import pandapower as pp

#create empty net
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")

#create bus elements
pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, name="Grid Connection")
pp.create_load(net, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")

#create branch elements
trafo = pp.create_transformer(net, hv_bus=bus1, lv_bus=bus2, std_type="0.4 MVA 20/0.4 kV", name="Trafo")
line = pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=0.1, std_type="NAYY 4x50 SE", name="Line")

## Data Structure

Each dataframe in a pandapower net object contains the information about one pandapower element, such as line, load transformer etc.

In [ ]:
net.bus

In [ ]:
net.line

In [ ]:
net.trafo

In [ ]:
net.load

Note that line and transformer are created with standard types, so thath the electric parameters of are automatically filled in from the standard type library.

## Power Flow

We now run a power flow:

In [ ]:
pp.runpp(net)

And check out at the results for buses, lines an transformers:

In [ ]:
net.res_bus

In [ ]:
net.res_line

In [ ]:
net.res_trafo

### Tap Changers

We now lower the tap changer position, from position 0 to -1 and run another power flow:

In [ ]:
net.trafo.tap_pos.at[trafo] = -1
pp.runpp(net)

Looking at the results shows that bus voltages at the low voltage side of the transformer have increased:

In [ ]:
net.res_bus

### Switches

We now create an open switch at the load bus:

In [ ]:
pp.create_switch(net, bus=bus3, element=line, et="l", closed=False)

The open switch cuts the load bus from power supply:

<img src="pics/3bus-system_switch.png" width="8%">

This can be verified by running a power flow and inspecting the results. The voltage at bus 2 is given as NaN:

In [ ]:
pp.runpp(net)
net.res_bus

The load does not feed in:

In [ ]:
net.res_load

And the line is in open loop operation:

In [ ]:
net.res_line

## Topological Analysis

The structure of the network can also be directly analyzed with the topology package. It uses an interface to the NetworkX library for graph searches. There are some predefined search algorithms, such as searching for unsupplied buses:

In [ ]:
import pandapower.topology as top
top.unsupplied_buses(net)

The package correctly determines that bus 2 is cut from power supply. When we close the switch, there are no unsupplied buses anymore:

In [ ]:
net.switch.closed.at[0] = True
top.unsupplied_buses(net)

Apart from predefined search functions, it is also possible to translate the pandapower network into a NetworkX graph and run searches directly on that graph.

Suppose we want to find all buses that are on the same voltage level as the load bus. We then translate the grid into a graph but excluding the transformer:

In [ ]:
mg = top.create_nxgraph(net, include_trafos=False)

And search for all buses that are connected to the load bus in that graph:

In [ ]:
list(top.connected_component(mg, 2))

The graph search finds all buses that are on the same voltage level. Searches like these can be used for feeder identification and many more applications.

## Short Circuit Analysis

pandapower includes a short circuit module that complies with IEC 60909. To run a short circuit analysis, we need to define short circuit parameters for the external grid:

In [ ]:
net.ext_grid["s_sc_max_mva"] = 100
net.ext_grid["rx_max"] = 0.1

Now we can calculate short circuits. Here, we calculate a three phase short circuit current with a fault impedance of 2 Ohms:

In [ ]:
import pandapower.shortcircuit as sc
sc.calc_sc(net, case="max", ip=True, r_fault_ohm=2.)

Initial and peak short circuit currents are given for faults at all buses:

In [ ]:
net.res_bus_sc

This concludes a short walkthrough of some pandapower features. More in-depth tutorials can be found in the pandapower documentation:
https://www.pandapower.org/start/#interactive-tutorials-